In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from google.colab import files

TIME_STEPS = 50
epochs = 20
batch_size = 64

In [ ]:
def load_and_process_data(filepath, scaler=None, fit_scaler=False):
    print(f"Loading {filepath}...")
    df = pd.read_csv(filepath)

    df['dx'] = df['x'].diff().fillna(0)
    df['dy'] = df['y'].diff().fillna(0)

    data = df[['dx', 'dy']].values

    if fit_scaler:
        scaler = MinMaxScaler()
        data_scaled = scaler.fit_transform(data)
    else:
        data_scaled = scaler.transform(data)

    X = []
    for i in range(len(data_scaled) - TIME_STEPS):
        X.append(data_scaled[i:(i + TIME_STEPS)])

    return np.array(X), scaler

In [ ]:
X_train, scaler = load_and_process_data('/content/golden_human.csv', fit_scaler=True)
joblib.dump(scaler, 'scaler.save')
print(f"Human Data Shape: {X_train.shape}")

model = Sequential([
    LSTM(32, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False),
    RepeatVector(X_train.shape[1]),
    LSTM(32, activation='relu', return_sequences=True),
    TimeDistributed(Dense(X_train.shape[2]))
])
model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, shuffle=True)
model.save('aimbot_hunter_model.h5')

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
def load_data_for_colab(filename, scaler):
    path = f"./{filename}" 
    try:
        df = pd.read_csv(path)
        df['dx'] = df['x'].diff().fillna(0)
        df['dy'] = df['y'].diff().fillna(0)
        data = df[['dx', 'dy']].values
        data_scaled = scaler.transform(data)
        
        X = []
        for i in range(len(data_scaled) - TIME_STEPS):
            X.append(data_scaled[i:(i + TIME_STEPS)])
        return np.array(X)
    except FileNotFoundError:
        print(f"⚠️ Could not find {filename}, skipping this data...")
        return None

def calculate_loss(model, data):
    reconstructions = model.predict(data)
    return np.mean(np.abs(reconstructions - data), axis=(1, 2))

print(">>> Judging...")

# model = tf.keras.models.load_model('aimbot_hunter_model.h5')
# scaler = joblib.load('scaler.save')

scores = {}
human_X = load_data_for_colab('golden_human.csv', scaler)
linear_X = load_data_for_colab('golden_linear.csv', scaler)
bezier_X = load_data_for_colab('golden_bezier.csv', scaler)

if human_X is not None: scores['Human'] = calculate_loss(model, human_X)
if linear_X is not None: scores['Linear Bot'] = calculate_loss(model, linear_X)
if bezier_X is not None: scores['Bezier Bot'] = calculate_loss(model, bezier_X)

plt.figure(figsize=(12, 6))
for label, score in scores.items():
    sns.histplot(score, bins=50, alpha=0.6, label=label, kde=True)

plt.axvline(x=0.05, color='r', linestyle='--', label='Threshold')
plt.title('Final Verdict: Human vs Aimbots')
plt.xlabel('Reconstruction Error (Loss)')
plt.legend()

plt.savefig('final_verdict.png')
plt.show()

print(">>> Packing and downloading critical files...")
files.download('aimbot_hunter_model.h5')
files.download('scaler.save')
files.download('final_verdict.png')